*this notebook uses a venv created by using uv*
- https://docs.astral.sh/uv/guides/integration/jupyter/#using-jupyter-from-vs-code

In [21]:
import pandas as pd
import torch
print(f"PyTorch version being used is: {torch.__version__}")
import torch.nn as nn

PyTorch version being used is: 2.2.2


In [2]:
# Consider also adding the moderate CYP3A4 substrates on top of the strong substrates 
# (otherwise data size is really really small...)
# also, may eventually merge into one ADRs column 
# (so it'll contain common, less common and postmarketing reported ADRs all in one column)

In [3]:
data = pd.read_csv("CYP3A4_strong_substrates")
data

,generic_drug_name,notes,cyp_strength_of_evidence,drug_class,common_adverse_effects,less_common_adverse_effects,first_ref,second_ref,date_checked
0,carbamazepine,NaN,strong,antiepileptics,"constipation^^, leucopenia^^, dizziness^^, som...","eosinophilia^, thrombocytopenia^, neutropenia^...",drugs.com,nzf,211024
1,eliglustat,NaN,strong,metabolic_agents,"diarrhea^^, oropharyngeal_pain^^, arthralgia^^...","rash^, flatulence^, dyspepsia^, gastroesophage...",drugs.com,emc,151124
2,flibanserin,NaN,strong,CNS_agents,"dizziness^^, somnolence^^","sedation^, fatigue^, vertigo^, accidental_inju...",drugs.com,Drugs@FDA,161124
3,imatinib,NaN,strong,tyrosine_kinase_inhibitor,"rash^^, diarrhea^^, abdominal_pain^^, constipa...","flushing^, pruritus^, face_edema^, dry skin^, ...",drugs.com,nzf,181124
4,ibrutinib,NaN,strong,tyrosine_kinase_inhibitor,"hypertension^^, atrial_fibrillation^^, sinus_t...","atrial_flutter^, cardiac_failure(pm)^, ventric...",drugs.com,nzf,191124
5,neratinib,NaN,strong,tyrosine_kinase_inhibitor,"diarrhea^^, abdominal_pain^^, stomatitis^^, dy...","abdominal_distention^, dry_mouth^, nail_disord...",drugs.com,nzf,201124
6,esomeprazole,NaN,strong,proton_pump_inhibitors,"headache^^, flatulence^^","dizziness^, somnolence^, taste_disturbance/per...",drugs.com,emc,161124
7,omeprazole,NaN,strong,proton_pump_inhibitors,"fever^^, otitis_media^^, respiratory_system_re...","accidental_injury^, asthenia^, pain(pm), fatig...",drugs.com,nzf,181124
8,ivacaftor,NaN,strong,CFTR_potentiator,"rash^^, oropharyngeal_pain^^, abdominal_pain^^...","acne^, increased_hepatic_enzymes^, increased_b...",drugs.com,nzf,201124
9,naloxegol,NaN,strong,peripheral_opioid_receptor_antagonists,abdominal pain^^,"possible_opioid_withdrawal_syndrome^, diarrhea...",drugs.com,emc,211124


For drug with astericks marked in "notes" column, see data notes under "Exceptions for ADRs" section in 1_ADR_data.qmd.

In [4]:
# drop some columns
df = data.drop([
    "cyp_strength_of_evidence", 
    "drug_class", 
    "less_common_adverse_effects", 
    "first_ref", 
    "second_ref", 
    "date_checked"
    ], axis=1)
df

,generic_drug_name,notes,common_adverse_effects
0,carbamazepine,NaN,"constipation^^, leucopenia^^, dizziness^^, som..."
1,eliglustat,NaN,"diarrhea^^, oropharyngeal_pain^^, arthralgia^^..."
2,flibanserin,NaN,"dizziness^^, somnolence^^"
3,imatinib,NaN,"rash^^, diarrhea^^, abdominal_pain^^, constipa..."
4,ibrutinib,NaN,"hypertension^^, atrial_fibrillation^^, sinus_t..."
5,neratinib,NaN,"diarrhea^^, abdominal_pain^^, stomatitis^^, dy..."
6,esomeprazole,NaN,"headache^^, flatulence^^"
7,omeprazole,NaN,"fever^^, otitis_media^^, respiratory_system_re..."
8,ivacaftor,NaN,"rash^^, oropharyngeal_pain^^, abdominal_pain^^..."
9,naloxegol,NaN,abdominal pain^^


In [5]:
string = df["generic_drug_name"].tolist()
# Convert list of drugs into multiple strings of drug names
drugs = f"'{"','".join(string)}'"
# Convert from lower case to upper case
for letter in drugs:
    if letter.islower():
        drugs = drugs.replace(letter, letter.upper())
print(drugs)

'CARBAMAZEPINE','ELIGLUSTAT','FLIBANSERIN','IMATINIB','IBRUTINIB','NERATINIB','ESOMEPRAZOLE','OMEPRAZOLE','IVACAFTOR','NALOXEGOL','OXYCODONE','SIROLIMUS','TERFENADINE'


In [6]:
# Get SMILES for each drug (via copying-and-pasting the previous cell output - attempted various ways to feed the string
# directly into cyp_drugs.py, current way seems to be the most straightforward one...)
from cyp_drugs import chembl_drugs
# Using ChEMBL version 34
df_s = chembl_drugs(
    'CARBAMAZEPINE','ELIGLUSTAT','FLIBANSERIN','IMATINIB','IBRUTINIB','NERATINIB','ESOMEPRAZOLE','OMEPRAZOLE','IVACAFTOR','NALOXEGOL','OXYCODONE','SIROLIMUS','TERFENADINE', 
    file_name="strong_cyp3a4_smiles"
    )
df_s

## Note: latest ChEMBL version 35 (as from 1st Dec 2024) seems to be taking a long time to load (no output after ~7min), 
## both versions 33 & 34 are ok with outputs loading within a few secs

,chembl_id,pref_name,max_phase,canonical_smiles
0,CHEMBL108,CARBAMAZEPINE,4,NC(=O)N1c2ccccc2C=Cc2ccccc21
1,CHEMBL2110588,ELIGLUSTAT,4,CCCCCCCC(=O)N[C@H](CN1CCCC1)[C@H](O)c1ccc2c(c1...
2,CHEMBL1201320,ESOMEPRAZOLE,4,COc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1
3,CHEMBL231068,FLIBANSERIN,4,O=c1[nH]c2ccccc2n1CCN1CCN(c2cccc(C(F)(F)F)c2)CC1
4,CHEMBL1873475,IBRUTINIB,4,C=CC(=O)N1CCC[C@@H](n2nc(-c3ccc(Oc4ccccc4)cc3)...
5,CHEMBL941,IMATINIB,4,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...
6,CHEMBL2010601,IVACAFTOR,4,CC(C)(C)c1cc(C(C)(C)C)c(NC(=O)c2c[nH]c3ccccc3c...
7,CHEMBL2219418,NALOXEGOL,4,C=CCN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](OCCOCC...
8,CHEMBL180022,NERATINIB,4,CCOc1cc2ncc(C#N)c(Nc3ccc(OCc4ccccn4)c(Cl)c3)c2...
9,CHEMBL1503,OMEPRAZOLE,4,COc1ccc2[nH]c([S+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1


I'm parsing the canonical SMILES through my old script to generate these small molecules as RDKit molecules and standardised SMILES, making sure these SMILES are valid and parsable.

In [8]:
# Using my previous code to preprocess small mols
import datamol as dm

# disable rdkit messages
dm.disable_rdkit_log()

#  The following function code were adapted from datamol.io
def preprocess(row):

    """
    Function to preprocess, fix, standardise, sanitise compounds 
    and then generate various molecular representations based on these molecules.
    Can be utilised as df.apply(preprocess, axis=1).

    :param smiles_column: SMILES column name (needs to be names as "canonical_smiles") 
    derived from ChEMBL database (or any other sources) via an input dataframe
    :param mol: RDKit molecules
    :return: preprocessed RDKit molecules, standardised SMILES, SELFIES, 
    InChI and InChI keys added as separate columns in the dataframe
    """

    # smiles_column = strings object
    smiles_column = "canonical_smiles"
    # Convert each compound into a RDKit molecule in the smiles column
    mol = dm.to_mol(row[smiles_column], ordered=True)
    # Fix common errors in the molecules
    mol = dm.fix_mol(mol)
    # Sanitise the molecules 
    mol = dm.sanitize_mol(mol, sanifix=True, charge_neutral=False)
    # Standardise the molecules
    mol = dm.standardize_mol(
        mol,
        # Switch on to disconnect metal ions
        disconnect_metals=True,
        normalize=True,
        reionize=True,
        # Switch on "uncharge" to neutralise charges
        uncharge=True,
        # Taking care of stereochemistries of compounds
        # Note: this uses the older approach of "AssignStereochemistry()" from RDKit
        # https://github.com/datamol-io/datamol/blob/main/datamol/mol.py#L488
        stereo=True,
    )

    # Adding following rows of different molecular representations 
    row["rdkit_mol"] = dm.to_mol(mol)
    row["standard_smiles"] = dm.standardize_smiles(dm.to_smiles(mol))
    #row["selfies"] = dm.to_selfies(mol)
    #row["inchi"] = dm.to_inchi(mol)
    #row["inchikey"] = dm.to_inchikey(mol)
    return row

df_s3a4 = df_s.apply(preprocess, axis = 1)
df_s3a4

,chembl_id,pref_name,max_phase,canonical_smiles,rdkit_mol,standard_smiles
0,CHEMBL108,CARBAMAZEPINE,4,NC(=O)N1c2ccccc2C=Cc2ccccc21,<rdkit.Chem.rdchem.Mol object at 0x12ad2c6d0>,NC(=O)N1c2ccccc2C=Cc2ccccc21
1,CHEMBL2110588,ELIGLUSTAT,4,CCCCCCCC(=O)N[C@H](CN1CCCC1)[C@H](O)c1ccc2c(c1...,<rdkit.Chem.rdchem.Mol object at 0x12ad2c7b0>,CCCCCCCC(=O)N[C@H](CN1CCCC1)[C@H](O)c1ccc2c(c1...
2,CHEMBL1201320,ESOMEPRAZOLE,4,COc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1,<rdkit.Chem.rdchem.Mol object at 0x12ad2c820>,COc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1
3,CHEMBL231068,FLIBANSERIN,4,O=c1[nH]c2ccccc2n1CCN1CCN(c2cccc(C(F)(F)F)c2)CC1,<rdkit.Chem.rdchem.Mol object at 0x12ad2c900>,O=c1[nH]c2ccccc2n1CCN1CCN(c2cccc(C(F)(F)F)c2)CC1
4,CHEMBL1873475,IBRUTINIB,4,C=CC(=O)N1CCC[C@@H](n2nc(-c3ccc(Oc4ccccc4)cc3)...,<rdkit.Chem.rdchem.Mol object at 0x12ad2c580>,C=CC(=O)N1CCC[C@@H](n2nc(-c3ccc(Oc4ccccc4)cc3)...
5,CHEMBL941,IMATINIB,4,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,<rdkit.Chem.rdchem.Mol object at 0x12ad2c510>,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...
6,CHEMBL2010601,IVACAFTOR,4,CC(C)(C)c1cc(C(C)(C)C)c(NC(=O)c2c[nH]c3ccccc3c...,<rdkit.Chem.rdchem.Mol object at 0x12ad2cb30>,CC(C)(C)c1cc(C(C)(C)C)c(NC(=O)c2c[nH]c3ccccc3c...
7,CHEMBL2219418,NALOXEGOL,4,C=CCN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](OCCOCC...,<rdkit.Chem.rdchem.Mol object at 0x12ad2cba0>,C=CCN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](OCCOCC...
8,CHEMBL180022,NERATINIB,4,CCOc1cc2ncc(C#N)c(Nc3ccc(OCc4ccccn4)c(Cl)c3)c2...,<rdkit.Chem.rdchem.Mol object at 0x12ad2c9e0>,CCOc1cc2ncc(C#N)c(Nc3ccc(OCc4ccccn4)c(Cl)c3)c2...
9,CHEMBL1503,OMEPRAZOLE,4,COc1ccc2[nH]c([S+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1,<rdkit.Chem.rdchem.Mol object at 0x12ad2ccf0>,COc1ccc2[nH]c([S+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1


In [9]:
# Splitting data 
# random splits usually lead to overly optimistic models... testing molecules are too similar to traininig molecules
# Some blog references re. data splitting wrt small molecules: 
# https://greglandrum.github.io/rdkit-blog/posts/2024-05-31-scaffold-splits-and-murcko-scaffolds1.html
# https://practicalcheminformatics.blogspot.com/2024/11/some-thoughts-on-splitting-chemical.html

## Try using Pat Walters' useful_rdkit_utils' GroupKFoldShuffle 
# (code originated from: https://github.com/scikit-learn/scikit-learn/issues/20520)

from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
import useful_rdkit_utils as uru
import numpy as np

# Generate numpy arrays containing the fingerprints 
df_s3a4['fp'] = df_s3a4.rdkit_mol.apply(rdFingerprintGenerator.GetMorganGenerator().GetCountFingerprintAsNumPy)

# Get Butina cluster labels
df_s3a4["butina_cluster"] = uru.get_butina_clusters(df_s3a4.standard_smiles)

# Set up a GroupKFoldShuffle object
group_kfold_shuffle = uru.GroupKFoldShuffle(n_splits=5, shuffle=True)

# Using cross-validation/doing data split
## X = np.stack(df_s3a4.fp), y = df.common_adverse_effects, group labels = df_s3a4.butina_cluster
for train, test in group_kfold_shuffle.split(np.stack(df_s3a4.fp), df.common_adverse_effects, df_s3a4.butina_cluster):
    print(len(train),len(test))

10 3
10 3
10 3
11 2
11 2


In [27]:
# butina cluster labels vs. index
df_s3a4["butina_cluster"]

0     10
1      9
2      1
3      8
4      7
5      6
6      5
7      0
8      4
9      1
10     0
11     3
12     2
Name: butina_cluster, dtype: int64

In [10]:
train

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [11]:
test

array([0, 1])

In [17]:
type(train)

numpy.ndarray

In [ ]:
# Transform train, test back into pd.DataFrame using iloc 
# (preparing for next step to convert adrs and drug_class columns into numericals)

# create a dictionary as {index: butina label} first?

In [22]:
# or maybe can directly convert from numpy to tensor
torch_train = torch.from_numpy(train)
torch_train

tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [23]:
torch_test = torch.from_numpy(test)
torch_test

tensor([0, 1])

In [13]:
## Likely using regression 
## convert ADRs into embedding (numericals) 
## convert drug_class column into one-hot encoding (numericals)
## if adding moderate CYP3A4 substrates, add "cyp_strength_of_evidence" column into training/testing sets,
## meaning to convert "cyp_strength_of_evidence" into one-hot encoding (numericals e.g. strong = 1, mod = 2)

# Use one-hot encoding for drug_class column
#from torch.nn.functional import one_hot


In [14]:
#from torch.utils.data import TensorDataset, DataLoader

## Create a PyTorch dataset (reference code below)
# training_data = TensorDataset(X_train, y_train)
# torch.manual_seed(1)
# batch_size = 2

## Create a dataset loader - DataLoader (reference code below)
# train_dataloader = DataLoader(training_data, batch_size, shuffle = True)

In [15]:
## Set up a DNN regression model 

In [16]:
# May need to set up a class with a few different functions (possibly in separate .py scripts then run in notebook first)

* Structure-adverse drug reaction relationships: 
**ADRs <-> (dense vectors of real numbers) <-> 2D drug structures**

* Structure-activity relationships: 
**drug activities <-> 2d drug structures**

1. building a NN model (?RNN or DNN initially) to classify drugs in an ADRs dataset (?identify drugs in different therapeutic classes) or to predict ADRs of drugs (regression) - to determine whether to use classification/regression (likely current post)
- to infer possible drugs vs. ADRs relationships

2. 2D drug structures part (much further down the line as a separate post)
- graph neural networks (GNN - other variations also available): molecules as undirected graphs where the connections between nodes (atoms) and edges (bonds) don't matter (i.e. don't need to be in particular orders or sequences) 
OR 
- RNN that uses SMILES (NLP technique) -> tokenize SMILES strings -> converts into a dictionary mapping tokens to indices in the vocabulary -> converts the vocabulary (SMILES strings) into one-hot encodings